In [13]:
%matplotlib inline
import json
import numpy as np
import pandas as pd
import plotly.offline as ply
#import rapidjson
#import ujson
import plotly.graph_objs as go
import datetime
from plotly import tools
from time import time, mktime
ply.init_notebook_mode(connected=True)

In [14]:
start = time()
#data = json.load(open('arbitrage-logger-export.json'))['log_btc_usd']
data = json.load(open('log_today.json'))['log_btc_usd']
print('Execution time: {:.3f}'.format(time() - start))

Execution time: 11.088


In [29]:
'''
it = iter(data.keys())
for i in range(5):
    ts = next(it)
    cur_asks = data[ts]['orders']['asks']
    cur_bids = data[ts]['orders']['bids']
    print('time: ', ts)
    print('asks: ', cur_asks)
    print('bids: ', cur_bids)
    print()
''';

In [37]:
start = time()


exchanges = {'bitstamp', 'cex', 'exmo', 'gdax'}
bids = {exch: 'bid_vol_' + exch for exch in exchanges}
asks = {exch: 'ask_vol_' + exch for exch in exchanges}
data_dict = {'timestamp': []}
for exch in exchanges:
    data_dict['bid_vol_' + exch] = []
    data_dict['ask_vol_' + exch] = []

i = 0
for key, val in data.items():
    if i == 200:
        break
    i += 1
    data_dict['timestamp'].append(key)
    bid_orders = val['orders']['bids']
    ask_orders = val['orders']['asks']
    for exch in exchanges:
        if exch in bid_orders:
            price = bid_orders[exch][0]
            amount = bid_orders[exch][1]
            data_dict[bids[exch]].append(float(price) * float(amount))
        else:
            data_dict[bids[exch]].append(0.0)
        
        if exch in ask_orders:
            price = ask_orders[exch][0]
            amount = ask_orders[exch][1]
            data_dict[asks[exch]].append(float(price) * float(amount))
        else:
            data_dict[asks[exch]].append(0.0)

df = pd.DataFrame(data_dict)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')


print('Execution time: {:.3f}'.format(time() - start))

Execution time: 0.008


In [38]:
df.head()

,ask_vol_bitstamp,ask_vol_cex,ask_vol_exmo,ask_vol_gdax,bid_vol_bitstamp,bid_vol_cex,bid_vol_exmo,bid_vol_gdax,timestamp
0,14773.30595,0.0,5882.538451,64603.200258,0.0,85309.681592,0.0,0.0,2018-05-02 02:47:10
1,14773.30595,0.0,5738.447737,49056.266843,0.0,69595.194026,0.0,0.0,2018-05-02 02:47:12
2,14773.30595,0.0,5792.863761,61861.747134,0.0,82438.716170,0.0,0.0,2018-05-02 02:47:13
3,14773.30595,0.0,5792.863761,61861.747134,0.0,82438.716170,0.0,0.0,2018-05-02 02:47:14
4,14773.30595,0.0,5792.863761,61861.747134,0.0,82438.716170,0.0,0.0,2018-05-02 02:47:16


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
ask_vol_bitstamp    200 non-null float64
ask_vol_cex         200 non-null float64
ask_vol_exmo        200 non-null float64
ask_vol_gdax        200 non-null float64
bid_vol_bitstamp    200 non-null float64
bid_vol_cex         200 non-null float64
bid_vol_exmo        200 non-null float64
bid_vol_gdax        200 non-null float64
timestamp           200 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(8)
memory usage: 14.1 KB


In [40]:
df.dropna(axis=0, how='any', inplace=True)  # drop rows with NA values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 9 columns):
ask_vol_bitstamp    200 non-null float64
ask_vol_cex         200 non-null float64
ask_vol_exmo        200 non-null float64
ask_vol_gdax        200 non-null float64
bid_vol_bitstamp    200 non-null float64
bid_vol_cex         200 non-null float64
bid_vol_exmo        200 non-null float64
bid_vol_gdax        200 non-null float64
timestamp           200 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(8)
memory usage: 15.6 KB


In [41]:
df.describe()

,ask_vol_bitstamp,ask_vol_cex,ask_vol_exmo,ask_vol_gdax,bid_vol_bitstamp,bid_vol_cex,bid_vol_exmo,bid_vol_gdax
count,200.000000,200.0,200.000000,200.000000,200.000000,200.000000,200.0,200.0
mean,9373.619370,0.0,5957.513482,49758.750276,392.328276,64715.995657,0.0,0.0
std,6112.127756,0.0,166.231787,12011.455898,2817.775867,13866.503442,0.0,0.0
min,0.000000,0.0,5183.327718,24423.733847,0.000000,42491.703055,0.0,0.0
25%,6674.220645,0.0,5897.788208,38538.929322,0.000000,50158.497526,0.0,0.0
50%,8939.506334,0.0,5938.219930,47433.607100,0.000000,63367.298977,0.0,0.0
75%,13008.021674,0.0,6027.942874,61699.241781,0.000000,76916.086220,0.0,0.0
max,25428.810807,0.0,6561.091338,70097.563912,23844.164657,86097.638000,0.0,0.0


In [43]:
start = time()


fig = tools.make_subplots(rows=4, cols=2, print_grid=False)
i = 1
for exch in exchanges:
    name = 'bid_vol_' + exch
    fig.append_trace(go.Scatter(x=df['timestamp'], y=df[name], name=name), i, 1)
    name = 'ask_vol_' + exch
    fig.append_trace(go.Scatter(x=df['timestamp'], y=df[name], name=name), i, 2)
    i += 1
ply.iplot(fig)


print('Execution time: {:.3f}'.format(time() - start))

Execution time: 0.182


In [47]:
start = time()

colors = {
    'bitstamp': 'rgba(255, 0, 0, 0.8)',
    'cex': 'rgba(0, 255, 0, 0.8)',
    'exmo': 'rgba(0, 0, 255, 0.8)',
    'gdax': 'rgba(0, 0, 0, 0.8)'
}

traces = []
for exch in exchanges:
    name = 'bid_vol_' + exch
    marker = dict(color = colors[exch])
    traces.append(go.Scatter(x=df['timestamp'], y=df[name], name=name, marker = marker))
ply.iplot(traces)


traces = []
for exch in exchanges:
    name = 'ask_vol_' + exch
    marker = dict(color = colors[exch])
    traces.append(go.Scatter(x=df['timestamp'], y=df[name], name=name, marker = marker))
ply.iplot(traces)


print('Execution time: {:.3f}'.format(time() - start))

Execution time: 0.262


In [45]:
'''
start = time()

colors = {
    'bitstamp': {'ask': 'rgba(255, 0, 0, 0.8)', 'bid': 'rgba(255, 0, 0, 0.4)'},
    'cex': {'ask': 'rgba(0, 255, 0, 0.8)', 'bid': 'rgba(0, 255, 0, 0.4)'},
    'exmo': {'ask': 'rgba(0, 0, 255, 0.8)', 'bid': 'rgba(0, 0, 255, 0.4)'},
    'gdax': {'ask': 'rgba(0, 0, 0, 0.8)', 'bid': 'rgba(255, 224, 229, 0.4)'},
}

traces = []
for exch in exchanges:
    name = 'bid_vol_' + exch
    marker = dict(color = colors[exch]['bid'])
    traces.append(go.Scatter(x=df['timestamp'], y=df[name], name=name, marker = marker))
ply.iplot(traces)


traces = []
for exch in exchanges:
    name = 'ask_vol_' + exch
    marker = dict(color = colors[exch]['ask'])
    traces.append(go.Scatter(x=df['timestamp'], y=df[name], name=name, marker = marker))
ply.iplot(traces)


print('Execution time: {:.3f}'.format(time() - start))
''';